In [52]:
#CREATE TABLE AND INSERT INTO TABLE
import sqlite3
connection=sqlite3.connect("agronomy.db")
crsr=connection.cursor()
sql_command="""CREATE TABLE WEATHER(WEATHER VARCHAR(20),LOW_HUMIDITY INT,HIGH_HUMIDITY INT,LOW_TEMPERATURE DECIMAL(4,2),HIGH_TEMPERATURE DECIMAL(4,2));"""
crsr.execute(sql_command)
sql_command="""INSERT INTO WEATHER VALUES('SUNNY',20,80,35,50);"""
crsr.execute(sql_command)
connection.commit()
connection.close()

OperationalError: table WEATHER already exists

In [53]:
#DISPLAY DETAILS IN WINDOW
from tkinter import *
import sqlite3
connection=sqlite3.connect("agronomy.db")
crsr=connection.cursor()
crsr.execute("select * from WEATHER;")
ans=crsr.fetchall()
root=Tk()
for i in ans:
    lab=Label(root,text=i,font=('Helvetica',8))
    lab.pack()
root.geometry("400x400+120+120")
root.mainloop()


In [94]:
#UPDATE TABLE
import sqlite3
connection=sqlite3.connect("agronomy.db")
crsr=connection.cursor()
crsr.execute("DELETE FROM USES where use='RICE';")
connection.commit()
print("Total number of changes made : ",connection.total_changes)
crsr.execute("select * from USES;")
ans=crsr.fetchall()
for i in ans:
    print(i)

Total number of changes made :  2
(1, 'PASTA')
(2, 'IDDLY')


In [9]:
#INSERT INTO TABLE WITH VARIABLE
import sqlite3
connection=sqlite3.connect("agronomy.db")
crsr=connection.cursor()
name=input("Weather name:")
lhum=int(input("lower humidity:"))
hhum=int(input("higher humidity:"))
ltem=int(input("lower temperature:"))
htem=int(input("higher temperature:"))
crsr.execute("insert into WEATHER values(?,?,?,?,?);",(name,lhum,hhum,ltem,htem))
crsr.execute("select * from WEATHER;")
ans=crsr.fetchall()
for i in ans:
    print(i)

Weather name: WINTER
lower humidity: 70
higher humidity: 98
lower temperature: -20
higher temperature: 25


('SUNNY', 15, 80, 35, 50)
('WINTER', 70, 98, -20, 25)


In [8]:
from tkinter import *
from tkinter import ttk
import sqlite3 
from tkinter import messagebox
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
def ends():
    root.destroy()
def endpage(f):
    f.place_forget()
    g=Frame(root,background="burlywood4")
    h=Label(g,text="Thank You!!",background="burlywood4",foreground="blue",font=("Comic Sans MS",50))
    h.grid(row=150,ipady=300)
    g.place(relx=0.5,rely=0.5,anchor="center")
    root.after(2000,ends)
    
def back(f,g):
    f.place_forget()
    g.place(relx=0.5,rely=0.5,anchor="center")
def browse(wlink,f,city,country):
    if(city!='' and country!=''):
        driver = webdriver.Chrome(r'C:\Users\Prashant\Downloads\Compressed\chromedriver')
        driver.get("https://www.wunderground.com/")
        searchkey=driver.find_element_by_id('wuSearch')
        searchkey.send_keys(city,",",country)
        def go():
            searchkey.send_keys(Keys.ENTER)
            global weatherlink
            weatherlink=driver.current_url
            driver.close()
        
            welink['text']="Successful!"
        root.after(15000,go)
        wlink.destroy()
        welink=Label(f,text="Please Wait....",background="burlywood4")
        welink.grid(row=8,column=1)
    else:
        messagebox.showerror("Incomplete City Field",message="Please complete the city field first!")
def des1():
    root.destroy()
    view()
def display(x):
    root=Tk()
    root.configure(background='burlywood4')
    crsr.execute(x)
    ans=crsr.fetchall()
    f=Frame(root)
    f.place(relx=0.5,rely=0.5,anchor="center")
    s=Scrollbar(f,orient=VERTICAL)
    lis=Listbox(f,height=20,width=120,yscrollcommand=s.set,font=("Calibri",18))
    s.config(command=lis.yview)
    s.pack(side=RIGHT,fill=Y)
    lis.pack(side=LEFT,fill=BOTH)
   
    for i in ans:
        lis.insert(END,i)
        #lab=Label(root,text=i,font=("Helvetica",12),background='burlywood4')
        #lab.pack()
    root.geometry("1900x2100")
    root.mainloop()
def ins_weather(zipcode):
    try:
        z=int(''.join(filter(lambda i: i.isdigit(),str(zipcode))))
        crsr.execute("select weather_link from area where zipcode=?;",(z,))
        welink=crsr.fetchone()
        welink=str(''.join(filter(lambda i:i,str(welink))))
        welink=welink.replace(',','')
        welink=welink.replace('(','')
        welink=welink.replace(')','')
        welink=welink.replace("'",'')
        request=requests.get(welink).text
        soup=BeautifulSoup(request,'lxml')
        article=soup.find_all('div',class_='columns small-12')[1]
        weather=article.find('a',class_='module-link').text
        #types=weather.split('.')[0]
        #temp=weather.split('.')[1]
        wind=weather.split('.')[2]
        article=soup.find_all('div',class_="condition-icon small-6 medium-12 columns")[0]
        types=article.find('p').text
        article=soup.find_all('div',class_="current-temp")[0]
        temp=article.find('span',class_='wu-value wu-value-to').text
        crsr.execute("update weather set `weather_type`=?,`temp`=?,`wind`=? where `zipcode`=?;",(types,temp,wind,z))
        connection.commit()
        
    except:
        messagebox.showerror("Sorry",message="Sorry could not get updated result.")
  
    
def ins_area(zcode,name,country,top):
    if(zcode!='' and name!='' and country!=''):
        try:
            crsr.execute("insert into area values(?,?,?,?);",(zcode,name,country,weatherlink))
            connection.commit()
            request=requests.get(weatherlink).text
            soup=BeautifulSoup(request,'lxml')
            article=soup.find_all('div',class_='columns small-12')[1]
            weather=article.find('a',class_='module-link').text
            #types=weather.split('.')[0]
            #temp=weather.split('.')[1]
            wind=weather.split('.')[2]
            article=soup.find_all('div',class_="condition-icon small-6 medium-12 columns")[0]
            types=article.find('p').text
            article=soup.find_all('div',class_="current-temp")[0]
            temp=article.find('span',class_='wu-value wu-value-to').text
            crsr.execute("insert into weather values(?,?,?,?);",(zcode,types,temp,wind))
            connection.commit()
            top.destroy()
            messagebox.showinfo("Successful!",message="Data successfully added!")
        except:
            messagebox.showinfo("Unsuccessful!",message="Zipcode should be unique!")
    else:
        messagebox.showerror("Error",message="Please complete the given entries!")
    admina()
def ins_cuses(cid,cuse,top):
    if(cid!='' and cuse!=''):
        top.destroy()                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
        crsr.execute("insert into USES values(?,?);",(cid,cuse))
        connection.commit() 
        messagebox.showinfo("Successful",message="Data successfully added!")
    else:
        messagebox.showerror("Error",message="Please Complete the entries!")
    c_uses(cid)
def selectp(cid,cname,capital,temp,ph,require,lis,f,top):
    f.place_forget()
    g=Frame(top,background="burlywood4")
    g.place(relx=0.5,rely=0.5,anchor="center")
    h=Frame(g,background="burlywood4")
    h.grid(row=0,column=1,pady=5,padx=5)
    pest=Label(g,text="Pesticide that can be used:",font=("Comic Sans MS",14),fg="maroon",background='burlywood4')
    s=Scrollbar(h,orient=VERTICAL)
    peslis=Listbox(h,selectmode="extend",height=5,width=15,yscrollcommand=s.set)
    s.config(command=peslis.yview)
    s.pack(side=RIGHT,fill=Y)
    peslis.pack()
    pest.grid(row=0,column=0,rowspan=5)
    crsr.execute("select pid,pname from pesticide;")
    p=crsr.fetchall()
    j=0
    for i in p:
               j=j+1
               peslis.insert(j,i[1])
    submit1=Button(g,text="submit",background='dark khaki',command=lambda:ins_crops(cid,cname,capital,temp,ph,require,lis,peslis.get(peslis.curselection()),f,top))
    submit1.grid(row=6,column=1,pady=10,ipadx=30,ipady=5)
def ins_crops(cid,cname,capital,temp,ph,require,lis,pest,f,top):
    if(cid!='' and cname!='' and capital!='' and temp!='' and ph!='' and pest!='' and len(require)!=0):
        #try:
        crsr.execute("select pid from pesticide where pname=?;",(pest,))
        pid=crsr.fetchone()
        pid=int(''.join(filter(lambda i: i.isdigit(),str(pid))))
        crsr.execute("insert into CROPS values(?,?,?,?,?,?);",(cid,cname,capital,temp,ph,pid))
        for i in require:
            ssn=lis.get(i)
            crsr.execute("INSERT INTO requirement VALUES(?,?,?,?);",(lowtemp[ssn],hightemp[ssn],ph,cid))
        connection.commit()
        top.destroy()
        c_uses(cid)
        #except:
            #messagebox.showinfo("Unsuccessful!",message="Crop ID should be unique!")
    else:
        messagebox.showerror("Error",message="Please complete the given entries!")
    adminc()
def ins_equip(eid,ename,cost,function,top):
    if(eid!='' and ename!='' and cost!='' and function!=''):
        try:
            top.destroy()
            crsr.execute("insert into EQUIPMENT values(?,?,?,?);",(eid,ename,cost,function))
            connection.commit()
            messagebox.showinfo("Successful",message="Data successfully added!")
        except:
            messagebox.showinfo("Unsuccessful",message="Equipment ID should be unique!")
    else:
        messagebox.showerror("Error",message="Please complete the given entries!")
    admine()
def ins_pesti(pid,pname,typ,amount,top):
    if(pid!='' and pname!='' and type!='' and amount!=''):
        try:
            top.destroy()
            crsr.execute("insert into PESTICIDE values(?,?,?,?);",(pid,pname,typ,amount))
            connection.commit()
            messagebox.showinfo("Successful",message="Data successfully added!")
        except:
            messagebox.showinfo("Unsuccessful",message="Pesticide ID should be unique!")
    else:
        messagebox.showerror("Error",message="Please complete the given entries!")
    adminp()

def admina():
    top=Toplevel()
    top.configure(background="burlywood4")
    f=Frame(top,background="burlywood4")
    title=Label(f,text="Location Details",font=("Matura MT Script Capitals",20),fg="maroon",background="burlywood4")
    zipcode_label=Label(f,text="Zip Code:",font=("Comic Sans MS",14),fg="maroon",background="burlywood4")
    name_label=Label(f,text="Area Name:",font=("Comic Sans MS",14),fg="maroon",background="burlywood4")
    country_label=Label(f,text="Country:",font=("Comic Sans MS",14),fg="maroon",background="burlywood4")
    link=Label(f,text="Link Weather: ",font=("Comic Sans MS",14),fg="maroon",background="burlywood4")
    wlink=Button(f,text="Get link",background="dark khaki",command=lambda:browse(wlink,f,name_entry.get(),country_entry.get()))
    zipcode_entry=Entry(f)
    name_entry=Entry(f)
    country_entry=Entry(f)
    title.grid(row=0,column=0,rowspan=2,columnspan=2,pady=20)
    zipcode_label.grid(row=2,column=0)
    name_label.grid(row=4,column=0)
    country_label.grid(row=6,column=0)
    zipcode_entry.grid(row=2,column=1)
    name_entry.grid(row=4,column=1)
    country_entry.grid(row=6,column=1)
    link.grid(row=8,column=0)
    wlink.grid(row=8,column=1,ipadx=20)
    submit1=Button(f,text="submit",background='dark khaki',command=lambda:ins_area(zipcode_entry.get(),name_entry.get(),country_entry.get(),top))
    submit1.grid(row=10,column=0,pady=10,ipadx=30,ipady=5)
    submit2=Button(f,text="display",background="dark khaki",command=lambda: display("select * from AREA;"))
    submit2.grid(row=10,column=1,pady=10,ipadx=30,ipady=5)
    submit3=Button(f,text="quit",background="dark khaki",command=top.destroy)
    submit3.grid(row=10,column=2,pady=10,ipadx=30,ipady=5)
    f.place(relx=0.5,rely=0.5,anchor="center")
    top.geometry("800x700+100+100")
def c_uses(cid):
    top=Toplevel()
    top.configure(background='burlywood4')
    f=Frame(top,background='burlywood4')
    title=Label(f,text="Used for producing",background='burlywood4',font=("Matura MT Script Capitals",20),fg="maroon")
    title.grid(row=0,column=0,rowspan=2,columnspan=3,padx=20,pady=20)
    cuses_label=Label(f,text="   Use:",background='burlywood4',fg="maroon")
    cuses_entry=Entry(f)
    cuses_label.grid(row=2,column=0,sticky="e")
    cuses_entry.grid(row=2,column=1)
    submit1=Button(f,text="Add",background='dark khaki',command=lambda:ins_cuses(cid,cuses_entry.get(),top))
    submit1.grid(row=3,column=0,pady=10,padx=10)
    submit2=Button(f,text="display",background='dark khaki',command=lambda: display("select * from USES;"))
    submit2.grid(row=3,column=1,padx=10,pady=10)
    submit3=Button(f,text="Cancel",background='dark khaki',command=top.destroy)
    submit3.grid(row=3,column=2,padx=10,pady=10)
    f.place(relx=0.5,rely=0.5,anchor="center")
    top.geometry("450x450+280+250")
    top.mainloop()

def adminc():
    top=Toplevel()
    top.configure(background='burlywood4')
    f=Frame(top,background='burlywood4')
    title=Label(f,text="Crop Details",font=("Matura MT Script Capitals",20),fg="maroon",background="burlywood4")
    cid_label=Label(f,text="Crop id:",font=("Comic Sans MS",14),fg="maroon",background='burlywood4')
    cropname_label=Label(f,text="Crop Name:",font=("Comic Sans MS",14),fg="maroon",background='burlywood4')
    capital_label=Label(f,text="Capital(Rs):",font=("Comic Sans MS",14),fg="maroon",background='burlywood4')
    temp_label=Label(f,text="Suitable temperature(F):",font=("Comic Sans MS",14),fg="maroon",background='burlywood4')
    ph_label=Label(f,text="Required Soil ph:",font=("Comic Sans MS",14),fg="maroon",background='burlywood4')
    season=Label(f,text="Favourable Season:",font=("Comic Sans MS",14),fg="maroon",background='burlywood4')
    cid_entry=Entry(f)
    cropname_entry=Entry(f)
    capital_entry=Entry(f)
    temp_entry=Entry(f)
    ph_entry=Entry(f)
    lis=Listbox(f,selectmode="multiple",height=5)
    lis.insert(0,"winter")
    lis.insert(1,"rainy")
    lis.insert(2,"autumn")
    lis.insert(3,"spring")
    lis.insert(4,"summer")
    lis.grid(row=12,column=1)
    season.grid(row=12,column=0)
    title.grid(row=0,column=0,rowspan=2,columnspan=4,pady=20)
    cid_label.grid(row=2,column=0)
    cropname_label.grid(row=4,column=0)
    capital_label.grid(row=6,column=0)
    temp_label.grid(row=8,column=0)
    ph_label.grid(row=10,column=0)
    cid_entry.grid(row=2,column=1)
    cropname_entry.grid(row=4,column=1)
    capital_entry.grid(row=6,column=1)
    temp_entry.grid(row=8,column=1)
    ph_entry.grid(row=10,column=1)
    submit1=Button(f,text="submit",background='dark khaki',command=lambda:selectp(cid_entry.get(),cropname_entry.get(),capital_entry.get(),temp_entry.get(),ph_entry.get(),lis.curselection(),lis,f,top))
    submit1.grid(row=18,column=0,pady=10,ipadx=30,ipady=5)
    submit2=Button(f,text="display",background='dark khaki',command=lambda: display("select * from CROPS;"))
    submit2.grid(row=18,column=1,pady=10,ipadx=30,ipady=5)
    submit3=Button(f,text="quit",background='dark khaki',command=top.destroy)
    submit3.grid(row=18,column=2,pady=10,ipadx=30,ipady=5)
    f.place(relx=0.5,rely=0.5,anchor="center")
    top.geometry("800x700+100+100")
def admine():
    top=Toplevel()
    top.configure(background="burlywood4")
    f=Frame(top,background="burlywood4")
    title=Label(f,text="Equipment Details",font=("Matura MT Script Capitals",20),fg="maroon",background="burlywood4")
    eid_label=Label(f,text="Equipment id:",font=("Comic Sans MS",14),background="burlywood4",fg="maroon")
    ename_label=Label(f,text="Equipment Name:",font=("Comic Sans MS",14),background="burlywood4",fg="maroon")
    cost_label=Label(f,text="Cost(Rs):",font=("Comic Sans MS",14),background="burlywood4",fg="maroon")
    function_label=Label(f,text="Function:",font=("Comic Sans MS",14),background="burlywood4",fg="maroon")
    eid_entry=Entry(f)
    ename_entry=Entry(f)
    cost_entry=Entry(f)
    function_entry=Entry(f)
    title.grid(row=0,column=0,rowspan=2,columnspan=4,pady=20)
    eid_label.grid(row=2,column=0)
    ename_label.grid(row=4,column=0)
    cost_label.grid(row=6,column=0)
    function_label.grid(row=8,column=0)
    eid_entry.grid(row=2,column=1)
    ename_entry.grid(row=4,column=1)
    cost_entry.grid(row=6,column=1)
    function_entry.grid(row=8,column=1)
    submit1=Button(f,text="submit",background="dark khaki",command=lambda:ins_equip(eid_entry.get(),ename_entry.get(),cost_entry.get(),function_entry.get(),top))
    submit1.grid(row=10,column=0,pady=10,ipadx=30,ipady=5)
    submit2=Button(f,text="display",background="dark khaki",command=lambda: display("select * from EQUIPMENT;"))
    submit2.grid(row=10,column=1,pady=10,ipadx=30,ipady=5)
    submit3=Button(f,text="quit",background="dark khaki",command=top.destroy)
    submit3.grid(row=10,column=2,pady=10,ipadx=30,ipady=5)
    f.place(relx=0.5,rely=0.5,anchor="center")
    top.geometry("800x700+100+100")
def adminp():
    top=Toplevel()
    top.configure(background="burlywood4")
    f=Frame(top,background="burlywood4")
    title=Label(f,text="Equipment Details",font=("Matura MT Script Capitals",20),fg="maroon",background="burlywood4")
    pid_label=Label(f,text="Pesticide id:",fg="maroon",font=("Comic Sans MS",14),background="burlywood4")
    pname_label=Label(f,text="Pesticide Name:",font=("Comic Sans MS",14),fg="maroon",background="burlywood4")
    type_label=Label(f,text="Type:",font=("Comic Sans MS",14),fg="maroon",background="burlywood4")
    amount_label=Label(f,text="Amount To Be Used(gm/100ltr):",font=("Comic Sans MS",14),fg="maroon",background="burlywood4")
    pid_entry=Entry(f)
    pname_entry=Entry(f)
    type_entry=Entry(f)
    amount_entry=Entry(f)
    title.grid(row=0,column=0,rowspan=2,columnspan=2,pady=20)
    pid_label.grid(row=2,column=0)
    pname_label.grid(row=4,column=0)
    type_label.grid(row=6,column=0)
    amount_label.grid(row=8,column=0)
    pid_entry.grid(row=2,column=1)
    pname_entry.grid(row=4,column=1)
    type_entry.grid(row=6,column=1)
    amount_entry.grid(row=8,column=1)
    submit1=Button(f,text="submit",background="dark khaki",command=lambda:ins_pesti(pid_entry.get(),pname_entry.get(),type_entry.get(),amount_entry.get(),top))
    submit1.grid(row=10,column=0,pady=10,ipadx=30,ipady=5)
    submit2=Button(f,text="display",background="dark khaki",command=lambda: display("select * from PESTICIDE;"))
    submit2.grid(row=10,column=1,pady=10,ipadx=30,ipady=5)
    submit3=Button(f,text="quit",background="dark khaki",command=top.destroy)
    submit3.grid(row=10,column=2,pady=10,ipadx=30,ipady=5)
    f.place(relx=0.5,rely=0.5,anchor="center")
    top.geometry("800x700+100+100")
    
def menu(h,username,password):
    if (username=='' or password==''):
        messagebox.showerror("Incomplete entry",message="Please complete the Entry field!")
    elif(dict["username"]==username and dict["password"]==password):
        h.place_forget()
        g=Frame(root,background="burlywood4")
        menut=Label(g,text="What do you want to Add?",font=("Matura MT Script Capitals",20),background="burlywood4")
        area=Button(g,text="Area",background="dark khaki",command=admina)
        crops=Button(g,text="Crops",background="dark khaki",command=adminc)
        equipments=Button(g,text="Equipments",background="dark khaki",command=admine)
        pesticides=Button(g,text="Pesticides",background="dark khaki",command=adminp)
        logout=Button(g,text="Log Out",background="dark khaki",command=lambda:back(g,f))
        menut.grid(row=0,column=0,rowspan=2,columnspan=4,padx=15,pady=15)
        area.grid(row=2,column=0,padx=15,pady=15,ipadx=30,ipady=10)
        crops.grid(row=2,column=1,padx=15,pady=15,ipadx=30,ipady=10)
        equipments.grid(row=4,column=0,padx=15,pady=15,ipadx=10,ipady=10)
        pesticides.grid(row=4,column=1,padx=15,pady=15,ipadx=20,ipady=10)
        logout.grid(row=5,column=1,pady=15,padx=15,ipadx=20,ipady=10)
        g.place(relx=0.5,rely=0.5,anchor="center")
    else:
        messagebox.showerror("Login Failed",message="Please enter valid username and password!!")
    
def change(f,ename):
    try:
        f.place_forget()
        crsr.execute("select * from equipment where ename=?;",(ename,))
        give=crsr.fetchone()
        g=Frame(root,background="burlywood4")
        g.place(relx=0.5,rely=0.5,anchor="center")
        l1=Label(g,text="Weather Based Agricultural Software",font=("Matura MT Script Capitals",20),background="burlywood4")
        l2=Label(g,text=ename+" Details",font=("Comic Sans MS bold",18),background="burlywood4")
        l3=Label(g,text="Equipment id: ",font=("Comic Sans MS",14),background="burlywood4")
        l4=Label(g,text="Equipment name: ",font=("Comic Sans MS",14),background="burlywood4")
        l5=Label(g,text="Cost: ",font=("Comic Sans MS",14),background="burlywood4")
        l6=Label(g,text="Function: ",font=("Comic Sans MS",14),background="burlywood4")
        l7=Label(g,text=str(give[0]),font=("Comic Sans MS",14),background="burlywood4")
        l8=Label(g,text=give[1],font=("Comic Sans MS",14),background="burlywood4")
        l9=Label(g,text="Rs"+str(give[2]),font=("Comic Sans MS",14),background="burlywood4")
        l10=Label(g,text=give[3],font=("Comic Sans MS",14),background="burlywood4")
        l1.grid(row=0,column=0,columnspan=2,rowspan=2,padx=20,pady=20)
        l2.grid(row=2,column=0,columnspan=2,rowspan=2,padx=10,pady=10)
        l3.grid(row=4,column=0,pady=10,sticky="e")
        l4.grid(row=6,column=0,pady=10,sticky="e")
        l5.grid(row=8,column=0,pady=10,sticky="e")
        l6.grid(row=10,column=0,pady=10,sticky="e")
        l7.grid(row=4,column=1,pady=10,sticky="w")
        l8.grid(row=6,column=1,pady=10,sticky="w")
        l9.grid(row=8,column=1,pady=10,sticky="w")
        l10.grid(row=10,column=1,pady=10,sticky="w")
        b1=Button(g,text="quit",background="dark khaki",command=lambda:endpage(g))
        b1.grid(row=20,column=1,ipadx=20,ipady=20,pady=5)
        b2=Button(g,text="<<Back",background="dark khaki",command=lambda:back(g,f))
        b2.grid(row=20,column=0,ipadx=20,ipady=20,pady=5) 
    except:
        messagebox.showerror("Sorry",message="We are not able to find any equipment!")
    
def checklocation(f,aname,country):
    crsr.execute("select zipcode from area where city=? and country=?;",(aname,country))
    ans=crsr.fetchone()
    row=0
    try:
        for i in ans:
            row=row+1
    except:
        messagebox.showinfo("error",message="Cannot find the given address, try again.")
    if(row>0):
        ins_weather(ans)
        f.place_forget()
        g=Frame(root,background="burlywood4")
        l1=Label(g,text="Weather Based Agricultural Software",font=("Matura MT Script Capitals",20),background="burlywood4")
        l2=Label(g,text="Enter the fertility(pH value) of your soil: ",font=("Comic Sans MS",14),background="burlywood4")
        e1=Entry(g)
        b1=Button(g,text="Next>>",background="dark khaki",command=lambda:checkrequirement(g,ans[0],e1.get()))
        b1.grid(row=8,column=1,ipadx=20,ipady=20,pady=5)
        b2=Button(g,text="<<Back",background="dark khaki",command=lambda:back(g,f))
        b2.grid(row=8,column=0,ipadx=20,ipady=20,pady=5)
        l1.grid(row=0,column=0,columnspan=2,rowspan=2,padx=20,pady=20)
        l2.grid(row=4,column=0,pady=10)
        e1.grid(row=4,column=1)
        
        g.place(relx=0.5,rely=0.5,anchor="center")
    #else:
     #   messagebox.showerror("Wrong Location",message="Cannot find the given address, try again.")
        
def suggestions(f,cname):
    if(cname!=''):
        try:
            crsr.execute("select pid,pname,type,amount from crops inner join pesticide on pid=pest_id where cname=?;",(cname,))
            pests=crsr.fetchone()
            f.place_forget()
            global sg
            sg=Frame(root,background="burlywood4")
            sg.place(relx=0.5,rely=0.5,anchor="center")
            l1=Label(sg,text="Weather Based Agricultural Software",font=("Matura MT Script Capitals",20),background="burlywood4")
            l2=Label(sg,text="Pesticide for "+cname,font=("Comic Sans MS bold",18),background="burlywood4")
            l3=Label(sg,text="Pesticide id: ",font=("Comic Sans MS",14),background="burlywood4")
            l4=Label(sg,text="Pesticide name: ",font=("Comic Sans MS",14),background="burlywood4")
            l5=Label(sg,text="Type: ",font=("Comic Sans MS",14),background="burlywood4")
            l6=Label(sg,text="Amount to be used: ",font=("Comic Sans MS",14),background="burlywood4")
            l7=Label(sg,text=pests[0],font=("Comic Sans MS",14),background="burlywood4")
            l8=Label(sg,text=pests[1],font=("Comic Sans MS",14),background="burlywood4")
            l9=Label(sg,text=pests[2],font=("Comic Sans MS",14),background="burlywood4")
            l10=Label(sg,text=str(pests[3])+" gm/100ltr",font=("Comic Sans MS",14),background="burlywood4")
            l11=Label(sg,text="Equipments that can be used:",font=("Comic Sans MS bold",18),background="burlywood4")
            l1.grid(row=0,column=0,columnspan=2,rowspan=2,padx=20,pady=20)
            l2.grid(row=2,column=0,columnspan=2,rowspan=2,padx=10,pady=10)
            l3.grid(row=4,column=0,pady=10,sticky="e")
            l4.grid(row=6,column=0,pady=10,sticky="e")
            l5.grid(row=8,column=0,pady=10,sticky="e")
            l6.grid(row=10,column=0,pady=10,sticky="e")
            l7.grid(row=4,column=1,pady=10,sticky="w")
            l8.grid(row=6,column=1,pady=10,sticky="w")
            l9.grid(row=8,column=1,pady=10,sticky="w")
            l10.grid(row=10,column=1,pady=10,sticky="w")
            l11.grid(row=11,column=0,pady=20,columnspan=2)
            global listequip 
            listequip=Listbox(sg,selectmode=SINGLE)
            crsr.execute("select eid,ename from equipment inner join utilize inner join crops on eqid=eid and crid=cid where cname=?;",(cname,))
            eq=crsr.fetchall()
            j=0
            for k in eq:
                listequip.insert(j,k[1])
                j=j+1
            
            listequip.grid(row=14,column=0)
            b1=Button(sg,text="Show",background="dark khaki",command=lambda:change(sg,listequip.get(listequip.curselection())))
            b1.grid(row=20,column=1,ipadx=20,ipady=20,pady=5)
            b2=Button(sg,text="Back",background="dark khaki",command=lambda:back(sg,f))
            b2.grid(row=20,column=0,ipadx=20,ipady=20,pady=5)

        except:
            messagebox.showerror("Sorry",message="No pesticide found.")
    else:
        messagebox.showerror("Error",message="Please select a crop.")
    
def checkrequirement(f,zipcode,p):
    ph=int(p)
    z=int(''.join(filter(lambda i: i.isdigit(),str(zipcode))))
    crsr.execute("select weather_type,temp,wind from weather where zipcode=?;",(z,))
    ans=crsr.fetchone()
    temp=int(''.join(filter(lambda i: i.isdigit(),ans[1])))
    try:
        crsr.execute("select distinct crid,cname from requirement inner join crops on crid=cid where hightemp>=? and lowtemp<=? and (requirement.soilph-?) in (1,0,-1);",(temp,temp,ph))
        res=crsr.fetchall()       
        f.place_forget()
        g=Frame(root,background="burlywood4")
        g.place(relx=0.5,rely=0.5,anchor="center")
        l1=Label(g,text="Weather Based Agricultural Software",font=("Matura MT Script Capitals",20),background="burlywood4")
        l2=Label(g,text="Weather: ",font=("Comic Sans MS bold",18),background="burlywood4")
        l3=Label(g,text="Temperature: ",font=("Comic Sans MS",14),background="burlywood4")
        l4=Label(g,text="Wind: ",font=("Comic Sans MS",14),background="burlywood4")
        l5=Label(g,text=ans[0],font=("Comic Sans MS bold",18),background="burlywood4")
        l6=Label(g,text=str(temp)+"F",font=("Comic Sans MS",14),background="burlywood4")
        l7=Label(g,text=ans[2],font=("Comic Sans MS",14),background="burlywood4")
       
        l1.grid(row=0,column=0,columnspan=2,rowspan=2,padx=20,pady=20)
        l2.grid(row=2,column=0,pady=10,sticky="e")
        l3.grid(row=4,column=0,pady=10,sticky="e")
        l4.grid(row=6,column=0,pady=10,sticky="e")
        l5.grid(row=2,column=1,pady=10,sticky="w")
        l6.grid(row=4,column=1,pady=10,sticky="w")
        l7.grid(row=6,column=1,pady=10,sticky="w")
        
        if(ph>8):
            messagebox.showinfo("Sorry",message="Your soil is highly basic. So, we don't have any crops suitable for them!")
        elif(ph<5):
            messagebox.showinfo("Sorry",message="Your soil is highly acidic. So, we don't have any crops suitable for them!")
        else:
            l8=Label(g,text="SELECT YOUR CROP",font=("Comic Sans MS bold",18),background="burlywood4")
            l8.grid(row=8,column=0,pady=20)
            listb=Listbox(g,selectmode=SINGLE)
            j=0
            for i in res:
                listb.insert(j,i[1])
                j=j+1
            
            listb.grid(row=9,column=0)
            b2=Button(g,text="Go>>",background="dark khaki",command=lambda:suggestions(g,listb.get(listb.curselection())))
            b2.grid(row=18,column=1,ipadx=20,ipady=20,pady=5)
        b1=Button(g,text="Home",background="dark khaki",command=lambda:user(g))
        b1.grid(row=18,column=0,ipadx=20,ipady=20,pady=5)
    except:
        messagebox.showinfo("Sorry",message="We could not find any crops suitable for you!")
        
    
def user(f):
    f.place_forget()
    g=Frame(root,background="burlywood4")
    l1=Label(g,text="Weather Based Agricultural Software",font=("Matura MT Script Capitals",20),background="burlywood4")
    l2=Label(g,text="Location/City",font=("Comic Sans MS",14),background="burlywood4")
    l3=Label(g,text="Country",font=("Comic Sans MS",14),background="burlywood4")
    e1=Entry(g)
    e2=Entry(g)
    b1=Button(g,text="Next>>",background="dark khaki",command=lambda:checklocation(g,e1.get(),e2.get()))
    b1.grid(row=8,column=1,ipadx=20,ipady=20,pady=5)
    l1.grid(row=0,column=0,columnspan=2,rowspan=2,padx=20,pady=20)
    l2.grid(row=4,column=0,pady=10)
    l3.grid(row=6,column=0,pady=10)
    e1.grid(row=4,column=1)
    e2.grid(row=6,column=1)
    g.place(relx=0.5,rely=0.5,anchor="center")
    

def login(f):
    f.place_forget()
    g=Frame(root,background="burlywood4")
    sub=Label(g,text="Login Page",font=("Comic Sans MS",18),background="burlywood4")
    sub.grid(row=2,column=0,columnspan=2,padx=10,pady=10)
    l1=Label(g,text="Username: ",background="burlywood4",font=("Comic Sans MS",14))
    l1.grid(row=3,column=0,padx=10,pady=10)
    l2=Label(g,text="Password: ",background="burlywood4",font=("Comic Sans MS",14))
    l2.grid(row=4,column=0,padx=10,pady=10)
    e1=Entry(g)
    e1.grid(row=3,column=1)
    e2=Entry(g,show="*")
    e2.grid(row=4,column=1)
    b=Button(g,text="Login",background="dark khaki",command=lambda:menu(g,e1.get(),e2.get()))
    b.grid(row=5,column=1,padx=10,pady=10,ipadx=5,ipady=5)
    g.place(relx=0.5,rely=0.5,anchor="center")

lowtemp={"winter":41,"rainy":60,"autumn":80,"spring":77,"summer":74}
hightemp={"winter":73,"rainy":80,"autumn":95,"spring":101,"summer":131}
dict={"username":"admin","password":"agronomy"}
connection=sqlite3.connect("agronomy.db")
crsr=connection.cursor()
root=Tk()
root.title('Agronomy')
f=Frame(root,background="burlywood4")
l=Label(f,text="Welcome to Agronomy",font=("Comic Sans MS",30),background="burlywood4")
admin=Button(f,text="ADMIN",background="dark khaki",command=lambda:login(f))
users=Button(f,text="USER",background="dark khaki",command=lambda:user(f))
l.grid(row=0,column=0,padx=40,pady=10,columnspan=2)
users.grid(row=3,column=1,padx=20,pady=10,ipadx=20,ipady=20)
admin.grid(row=3,column=0,padx=20,pady=10,ipadx=20,ipady=20)
f.place(relx=0.5,rely=0.5,anchor="center")
root.configure(background="burlywood4")
root.geometry("800x700+100+100")
root.mainloop()
connection.close()




In [2]:
import sqlite3
connection=sqlite3.connect("agronomy.db")
crsr=connection.cursor()
#sql_command="Create table requirement(lowtemp int, hightemp int, soilph int, crid int, constraint fk7 foreign key (crid) references crops(cid), check (soilph between 0 and 14));"
#crsr.execute(sql_command)
sql_command="INSERT INTO requirement VALUES(41,73,5,'4');"
crsr.execute(sql_command)
#sql_command="""SELECT * FROM ;"""
#crsr.execute(sql_command)
#ans=crsr.fetchall()
#for i in ans:
#    print(i)
connection.commit()
connection.close()

In [7]:
import sqlite3
connection=sqlite3.connect("agronomy.db")
crsr=connection.cursor()
sql_command="delete from area where city='udaipur';"
crsr.execute(sql_command)
connection.commit()
connection.close()
   

In [20]:
import sqlite3
connection=sqlite3.connect("agronomy.db")
crsr=connection.cursor()
sql_command="select * from weather;"
crsr.execute(sql_command)
ans=crsr.fetchall()
for i in ans:
    print(i)
connection.commit()
connection.close()
   

(560103, 'Foggy', ' Low 63F', ' Winds light and variable')
(403110, 'Partly cloudy skies', ' Low 77F', ' Winds light and variable')
(32907, 'Abundant sunshine', ' High 87F', ' Winds light and variable')
(44600, 'Mainly clear skies', ' Low 52F', ' Winds S at 5 to 10 mph')
(560809, 'Sunny', ' Expect widespread areas of smoke and haze, reducing visibility at times', ' High near 85F')
(34090, 'Clear skies', ' Expect widespread areas of smoke and haze, reducing visibility at times', ' Low 62F')
(56023, 'Morning fog, then mostly cloudy this afternoon', ' High 82F', ' Winds light and variable')
(45023, 'Partly cloudy with a slight chance of thunderstorms', ' Expect widespread areas of smoke and haze, reducing visibility at times', ' High 86F')
(54312, 'Sunny', ' Expect widespread areas of smoke and haze, reducing visibility at times', ' High around 85F')
(567109, 'Some sunshine with a thunderstorm or two possible this afternoon', ' Expect widespread areas of smoke and haze, reducing visibilit

In [78]:
from tkinter import *
import sqlite3
def view():
    root=Tk()
    root.title("AGRONOMY")
    f1=Frame(root)
    place=Label(f1,text="Enter Your Area Name:",font=(20))
    place.grid(row=0,column=0)
    placeentry=Entry(f1,width=30)
    placeentry.grid(row=0,column=1)  
    f1.place(x=10,y=10)
    f2=Frame(root)
    country=Label(f2,text="Country:                      ",font=(20))
    country.pack(side='left')
    countryentry=Entry(f2,width=30)
    countryentry.pack(side='right')
    f2.place(x=10,y=60)
    f3=Frame(root)
    humidity=Label(f3,text="Humidity")
    humidity.pack(side='left')
    hentry=Entry(f3,width=30)
    hentry.pack(side='right')
    f3.place(x=10,y=60)
    f4=Frame(root)
    f4.place(x=400,y=110)
    f5=Frame(root)
    submit=Button(f3,text="Search",font=(15),fg='green',command=lambda:predict(root,placeentry.get(),countryentry.get()))
    submit.pack(side='right')
    f5.place(x=400,y=110)
    root.geometry("600x600+120+120")
    root.mainloop()
    
def predict(root,place,country):
    f1=Frame(root)
    crsr.execute("select * from WEATHER;")
    ans=fetchall()
    crsr.execute("select * from AREA;")
    ans1=fetchall()
    for loc in ans1:
        if(loc[0]==place and loc[1]==country):
            for weat in ans:
                
                
                
                
                
                
    title=Label(root,text="It is ? in Butwal.",(weat[1]))

connection=sqlite3.connect("agronomy.db")    
crsr=connection.cursor()
view()
connection.close()

IndentationError: expected an indented block (<ipython-input-78-48e9f98bfa28>, line 48)

In [13]:
string='ut has 16letters and words.'
res=[int(i) for i in string.split() if i.isdigit()]
print(res) 

[]


In [88]:
string='it has 16l\/,,etters and words.'
res=str(''.join(filter(lambda i: i,string)))
res=res.replace(',','')
print(res)

it has 16l\/etters and words.


In [108]:
import sqlite3
conn=sqlite3.connect("agronomy.db")
crsr=conn.cursor()
temp=75
ph=6
crsr.execute("select crid,cname from requirement inner join crops on crid=cid where hightemp>=? and lowtemp<=? and (soilph-?) in (1,0,-1);",(temp,temp,ph))
res=crsr.fetchall()
for i in res:
    print(i)
conn.close()

(1, 'water melon')


In [ ]:
import sqlite3
connection=sqlite3.connect("agronomy.db")
crsr=connection.cursor()
sql_command="insert into utilize values(,);"
crsr.execute(sql_command)
ans=crsr.fetchall()
for i in ans:
    print(i)
connection.commit()
connection.close()
   